In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.feature_selection import RFE

from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [ ]:
# Link notebook with google drive and access data from your personal Gdrive
from google.colab import drive
#Mount Google Drive
drive.mount('/content/gdrive')

#Set the data path for dataset and model location (ex: model_loc = "/content/gdrive/My Drive/Dataset/")
dataset_dir = "/content/gdrive/My Drive/Bio Modelling/data/"
model_loc = "/content/gdrive/My Drive/Bio Modelling/data/"

print(os.listdir(dataset_dir))

Mounted at /content/gdrive
['BRCA_miRNA.csv', 'BRCA_label.csv', 'BRCA_DNA_Methylation.csv', 'BRCA_mRNA.csv', 'BRCA_Copy Nunber Variation.csv']


In [ ]:
data = pd.read_csv(dataset_dir+'BRCA_Copy Nunber Variation.csv',index_col=0)
data

,TCGA.3C.AAAU.01,TCGA.3C.AALI.01,TCGA.3C.AALJ.01,TCGA.3C.AALK.01,TCGA.5L.AAT0.01,TCGA.5T.A9QA.01,TCGA.A1.A0SB.01,TCGA.A1.A0SE.01,TCGA.A1.A0SF.01,TCGA.A1.A0SG.01,...,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01,TCGA.V7.A7HQ.01,TCGA.W8.A86G.01,TCGA.WT.AB41.01,TCGA.WT.AB44.01,TCGA.XX.A899.01,TCGA.XX.A89A.01,TCGA.Z7.A8R5.01,TCGA.Z7.A8R6.01
A1BG,0.495252,1.028904,0.003059,-0.258705,-0.083232,-0.486242,-0.111192,-0.102997,0.687116,-0.213873,...,-0.493956,2.097171,-0.594226,0.478862,-1.755051,-0.835261,-0.148311,0.557439,-0.234602,-0.838154
A1CF,-0.938762,-1.575977,0.405308,0.096910,0.060782,2.050968,0.154506,0.018894,1.131535,0.145081,...,-0.299975,6.501006,0.255559,-0.184784,0.371274,0.483323,0.061306,0.627836,0.070730,-0.463861
A2M,0.750838,0.010864,0.103201,-0.169130,-2.090765,-0.216362,-0.062751,-0.056368,0.857217,-0.016369,...,1.139760,-1.938856,0.036395,-0.308274,0.118520,0.266599,-0.089133,0.344469,0.226601,-0.590817
A2ML1,0.748593,0.005894,0.098571,-0.174763,-2.103474,-0.222169,-0.067992,-0.061585,0.855364,-0.021439,...,1.138948,-1.951005,0.031519,-0.314419,0.113946,0.262571,-0.094471,0.340728,0.222425,-0.598003
A3GALT2,1.093743,-1.575680,-0.706498,0.199712,0.376570,-2.334753,0.400443,0.225047,1.226264,0.390699,...,-2.541330,-1.587373,0.371698,0.096424,-1.494803,-2.569588,0.046241,-1.244865,-0.252418,-0.264599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,1.267331,-1.793233,-0.872324,0.058733,0.242915,-2.475162,0.263210,0.085117,1.176508,0.207398,...,-0.169592,0.192176,0.237841,-0.048833,-1.672982,-2.900353,-0.073695,-0.277665,-0.435463,-0.424808
ZYX,0.963246,1.446230,0.039191,-0.257130,-0.185487,-1.970238,-0.112381,-0.106533,0.704939,-0.262491,...,-0.542729,-0.489606,-0.134313,0.310169,-0.002235,1.155757,-0.286372,0.253634,-0.327799,-0.589030
ZZEF1,-0.580418,-0.790000,-1.421040,-1.377441,-0.999581,1.646581,0.848410,-0.727278,-0.501251,0.866003,...,-1.375911,-0.695152,0.920311,-0.919650,-0.690180,0.438043,0.811313,-0.470272,0.723732,-2.171404
ZZZ3,1.715811,-1.268621,-0.811368,0.098802,0.204322,-1.930651,0.307432,0.230822,-1.278740,0.290087,...,-2.491497,-0.100674,0.289605,-0.009609,-1.555790,-2.902013,-0.051046,-0.247150,-0.350742,-0.339178


In [ ]:
label = pd.read_csv(dataset_dir+'BRCA_label.csv')
label

,Label
0,LumA
1,Her2
2,LumB
3,LumA
4,LumA
...,...
666,LumA
667,LumA
668,LumA
669,LumA


In [ ]:
#replace label with numerical data
label['Label'].replace('Normal', 0 , inplace=True)
label['Label'].replace('LumA', 1 , inplace=True)
label['Label'].replace('LumB', 2 , inplace=True)
label['Label'].replace('Basal', 3 , inplace=True)
label['Label'].replace('Her2', 4 , inplace=True)

In [ ]:
# Transpose data to match with label
data = data. T
data

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk
TCGA.3C.AAAU.01,0.495252,-0.938762,0.750838,0.748593,1.093743,-0.876388,1.109460,-1.966983,-1.676420,1.090124,...,-0.837399,2.020624,1.972719,1.053381,1.254989,1.267331,0.963246,-0.580418,1.715811,-0.864183
TCGA.3C.AALI.01,1.028904,-1.575977,0.010864,0.005894,-1.575680,-0.967673,1.481901,-0.040373,-0.027748,1.213108,...,-1.537275,4.002085,3.897638,1.589442,-1.781406,-1.793233,1.446230,-0.790000,-1.268621,0.345052
TCGA.3C.AALJ.01,0.003059,0.405308,0.103201,0.098571,-0.706498,-1.643561,-0.661117,-1.342185,1.469986,-0.571499,...,4.165544,-0.319977,-0.301093,-0.477515,-0.867769,-0.872324,0.039191,-1.421040,-0.811368,-1.357237
TCGA.3C.AALK.01,-0.258705,0.096910,-0.169130,-0.174763,0.199712,0.570857,-0.422884,-0.156210,-0.017618,-0.402285,...,0.080744,-0.316675,-0.297884,-0.267586,0.055935,0.058733,-0.257130,-1.377441,0.098802,0.042601
TCGA.5L.AAT0.01,-0.083232,0.060782,-2.090765,-2.103474,0.376570,0.631333,-0.350660,-0.101592,0.100904,-0.312965,...,0.045801,-0.307593,-0.289062,-0.200167,0.238663,0.242915,-0.185487,-0.999581,0.204322,0.078553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44.01,-0.835261,0.483323,0.266599,0.262571,-2.569588,-1.733704,-0.412105,0.359954,0.516238,-0.364133,...,0.454483,2.317018,2.260655,-0.081225,-2.879783,-2.900353,1.155757,0.438043,-2.902013,0.247469
TCGA.XX.A899.01,-0.148311,0.061306,-0.089133,-0.094471,0.046241,0.592537,-0.391623,-0.094390,0.009733,-0.347077,...,0.046307,-0.281173,-0.263396,-0.182627,-0.075447,-0.073695,-0.286372,0.811313,-0.051046,-0.178246
TCGA.XX.A89A.01,0.557439,0.627836,0.344469,0.340728,-1.244865,-0.629158,-0.897733,0.600030,0.666163,-0.768542,...,0.594256,-0.279522,-0.261792,-0.686897,-0.277806,-0.277665,0.253634,-0.470272,-0.247150,-0.427626
TCGA.Z7.A8R5.01,-0.234602,0.070730,0.226601,0.222425,-0.252418,0.087808,-1.021162,0.488395,0.491419,-0.882100,...,0.012883,-0.345571,-0.325956,-0.844207,-0.434358,-0.435463,-0.327799,0.723732,-0.350742,0.076841


In [ ]:
# Ensure the number of rows in data match with label
if data.shape[0] == label.shape[0]:
    # Add the new column to the transposed DataFrame
    data['Label'] = label['Label'].values
else:
    raise ValueError("The number of rows in df2 must match the number of columns in the transposed df1.")


In [ ]:
data

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk,Label
TCGA.3C.AAAU.01,0.495252,-0.938762,0.750838,0.748593,1.093743,-0.876388,1.109460,-1.966983,-1.676420,1.090124,...,2.020624,1.972719,1.053381,1.254989,1.267331,0.963246,-0.580418,1.715811,-0.864183,1
TCGA.3C.AALI.01,1.028904,-1.575977,0.010864,0.005894,-1.575680,-0.967673,1.481901,-0.040373,-0.027748,1.213108,...,4.002085,3.897638,1.589442,-1.781406,-1.793233,1.446230,-0.790000,-1.268621,0.345052,4
TCGA.3C.AALJ.01,0.003059,0.405308,0.103201,0.098571,-0.706498,-1.643561,-0.661117,-1.342185,1.469986,-0.571499,...,-0.319977,-0.301093,-0.477515,-0.867769,-0.872324,0.039191,-1.421040,-0.811368,-1.357237,2
TCGA.3C.AALK.01,-0.258705,0.096910,-0.169130,-0.174763,0.199712,0.570857,-0.422884,-0.156210,-0.017618,-0.402285,...,-0.316675,-0.297884,-0.267586,0.055935,0.058733,-0.257130,-1.377441,0.098802,0.042601,1
TCGA.5L.AAT0.01,-0.083232,0.060782,-2.090765,-2.103474,0.376570,0.631333,-0.350660,-0.101592,0.100904,-0.312965,...,-0.307593,-0.289062,-0.200167,0.238663,0.242915,-0.185487,-0.999581,0.204322,0.078553,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44.01,-0.835261,0.483323,0.266599,0.262571,-2.569588,-1.733704,-0.412105,0.359954,0.516238,-0.364133,...,2.317018,2.260655,-0.081225,-2.879783,-2.900353,1.155757,0.438043,-2.902013,0.247469,1
TCGA.XX.A899.01,-0.148311,0.061306,-0.089133,-0.094471,0.046241,0.592537,-0.391623,-0.094390,0.009733,-0.347077,...,-0.281173,-0.263396,-0.182627,-0.075447,-0.073695,-0.286372,0.811313,-0.051046,-0.178246,1
TCGA.XX.A89A.01,0.557439,0.627836,0.344469,0.340728,-1.244865,-0.629158,-0.897733,0.600030,0.666163,-0.768542,...,-0.279522,-0.261792,-0.686897,-0.277806,-0.277665,0.253634,-0.470272,-0.247150,-0.427626,1
TCGA.Z7.A8R5.01,-0.234602,0.070730,0.226601,0.222425,-0.252418,0.087808,-1.021162,0.488395,0.491419,-0.882100,...,-0.345571,-0.325956,-0.844207,-0.434358,-0.435463,-0.327799,0.723732,-0.350742,0.076841,1


In [ ]:
# print the total number of data with missing values across all columns
print(data.isna().sum().sum())

0


In [ ]:
# print the total number of data with duplicated values across all columns
print(data.duplicated().sum())

0


In [ ]:
print(data["Label"].value_counts()[0])
print(data["Label"].value_counts()[1])
print(data["Label"].value_counts()[2])
print(data["Label"].value_counts()[3])
print(data["Label"].value_counts()[4])

31
353
132
113
42


In [ ]:
# Separate features and target
y = data['Label'].values
x = data.drop(['Label'],axis=1)

In [ ]:
# Normalization
scaler = MinMaxScaler(feature_range=(-1, 1))
X_normalized = scaler.fit_transform(x)

# Apply the scaler to the DataFrame
x = pd.DataFrame(scaler.fit_transform(X_normalized), index = x.index, columns = x.columns)
x

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk
TCGA.3C.AAAU.01,-0.172726,-0.304296,-0.076241,-0.048366,0.201374,-0.149514,0.087144,-0.419400,-0.585707,0.151653,...,-0.411802,0.206165,-0.012023,0.354060,0.157600,0.236162,-0.009469,-0.397939,0.321668,-0.302060
TCGA.3C.AALI.01,-0.090680,-0.416009,-0.233517,-0.201568,-0.377801,-0.172384,0.172075,0.026681,-0.248576,0.173088,...,-0.521860,0.526421,0.250301,0.461315,-0.545799,-0.514974,0.082529,-0.438278,-0.377547,-0.164382
TCGA.3C.AALJ.01,-0.248397,-0.068662,-0.213892,-0.182451,-0.189218,-0.341719,-0.316618,-0.274736,0.057690,-0.137962,...,0.374930,-0.172138,-0.321893,0.047760,-0.334150,-0.288961,-0.185481,-0.559735,-0.270418,-0.358196
TCGA.3C.AALK.01,-0.288642,-0.122728,-0.271774,-0.238834,0.007400,0.213074,-0.262291,-0.000139,-0.246504,-0.108469,...,-0.267421,-0.171604,-0.321456,0.089763,-0.120168,-0.060457,-0.241924,-0.551343,-0.057177,-0.198817
TCGA.5L.AAT0.01,-0.261664,-0.129062,-0.680203,-0.636684,0.045772,0.228226,-0.245821,0.012507,-0.222268,-0.092900,...,-0.272916,-0.170136,-0.320254,0.103252,-0.077838,-0.015254,-0.228277,-0.478616,-0.032455,-0.194724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44.01,-0.377284,-0.054984,-0.179163,-0.148621,-0.593446,-0.364303,-0.259833,0.119372,-0.137338,-0.101819,...,-0.208649,0.254070,0.027216,0.127049,-0.800245,-0.786688,0.027200,-0.201914,-0.760230,-0.175492
TCGA.XX.A899.01,-0.271669,-0.128970,-0.254771,-0.222271,-0.025899,0.218506,-0.255162,0.014175,-0.240911,-0.098846,...,-0.272836,-0.165866,-0.316756,0.106761,-0.150603,-0.092958,-0.247494,-0.130070,-0.092284,-0.223962
TCGA.XX.A89A.01,-0.163165,-0.029649,-0.162612,-0.132499,-0.306025,-0.087574,-0.370575,0.174958,-0.106680,-0.172306,...,-0.186669,-0.165599,-0.316537,0.005867,-0.197481,-0.143017,-0.144634,-0.376739,-0.138229,-0.252355
TCGA.Z7.A8R5.01,-0.284936,-0.127318,-0.187664,-0.156902,-0.090698,0.092053,-0.398722,0.149111,-0.142413,-0.192099,...,-0.278092,-0.176274,-0.325281,-0.025607,-0.233747,-0.181745,-0.255384,-0.146927,-0.162499,-0.194919


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
print(y_train.tolist().count(0))
print(y_train.tolist().count(1))
print(y_train.tolist().count(2))
print(y_train.tolist().count(3))
print(y_train.tolist().count(4))

In [ ]:
# Define the SVM model
model = SVC(kernel="linear")

# Define RFE with the SVM model, choosing a number of features to select
rfe = RFE(estimator=model, n_features_to_select=19000)

# Fit RFE on the training data
rfe.fit(X_train, y_train)

# Transform the training and test sets to include only the selected features
x_train_rfe = rfe.transform(X_train)
x_test_rfe = rfe.transform(X_test)

In [ ]:
# Convert the transformed arrays back to DataFrames
# Get the selected feature names
selected_features = X_train.columns[rfe.support_]

X_train_rfe = pd.DataFrame(x_train_rfe, columns=selected_features, index=X_train.index)
X_test_rfe = pd.DataFrame(x_test_rfe, columns=selected_features, index=X_test.index)

In [ ]:
# Define the model
model = Sequential()

# Add convolutional layer
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(19000, 1)))
model.add(MaxPooling1D(pool_size=2))

# Add more convolutional layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Add another convolutional layer
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(5, activation='softmax'))  # 5 classes

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_15 (Conv1D)          (None, 18998, 32)         128       
                                                                 
 max_pooling1d_15 (MaxPooli  (None, 9499, 32)          0         
 ng1D)                                                           
                                                                 
 conv1d_16 (Conv1D)          (None, 9497, 64)          6208      
                                                                 
 max_pooling1d_16 (MaxPooli  (None, 4748, 64)          0         
 ng1D)                                                           
                                                                 
 conv1d_17 (Conv1D)          (None, 4746, 128)         24704     
                                                                 
 max_pooling1d_17 (MaxPooli  (None, 2373, 128)        

In [ ]:
# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model with early stopping
model.fit(X_train_rfe, y_train, epochs=250, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
12/12 [==============================] - 3s 126ms/step - loss: 3.0988 - accuracy: 0.4427 - val_loss: 1.3363 - val_accuracy: 0.5213
Epoch 2/100
12/12 [==============================] - 1s 81ms/step - loss: 1.1682 - accuracy: 0.5973 - val_loss: 1.1152 - val_accuracy: 0.5638
Epoch 3/100
12/12 [==============================] - 1s 82ms/step - loss: 0.9348 - accuracy: 0.6507 - val_loss: 0.9177 - val_accuracy: 0.6702
Epoch 4/100
12/12 [==============================] - 1s 64ms/step - loss: 0.7692 - accuracy: 0.7333 - val_loss: 0.9226 - val_accuracy: 0.6064
Epoch 5/100
12/12 [==============================] - 1s 81ms/step - loss: 0.6568 - accuracy: 0.7653 - val_loss: 0.8888 - val_accuracy: 0.6277
Epoch 6/100
12/12 [==============================] - 1s 82ms/step - loss: 0.5872 - accuracy: 0.8080 - val_loss: 0.7732 - val_accuracy: 0.6383
Epoch 7/100
12/12 [==============================] - 1s 82ms/step - loss: 0.5085 - accuracy: 0.8373 - val_loss: 0.7521 - val_accuracy: 0.6915
Epoch

In [ ]:
# Predict probabilities
y_pred_prob = model.predict(X_test_rfe)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

7/7 [==============================] - 0s 21ms/step
Accuracy: 0.6831683168316832
